In [1]:
/* this is code for a project using the Seattle Library Checkout data from Kaggle
https://www.kaggle.com/seattle-public-library/seattle-library-checkout-records/data
*/

val c2015 = spark.read.option("inferSchema","true").option("header","true").csv("file:///data/checkout2015")
val c2016 = spark.read.option("inferSchema","true").option("header","true").csv("file:///data/checkout2016")
val c2017 = spark.read.option("inferSchema","true").option("header","true").csv("file:///data/checkout2017")
val ils = spark.read.option("inferSchema","true").option("header","true").csv("file:///data/ILS_data_dict")
val inventory = spark.read.option("inferSchema","true").option("header","true").csv("file:///data/lib_collection_inv")

val c2015DF = c2015.toDF
val c2016DF = c2016.toDF
val c2017DF = c2017.toDF
val ilsDF = ils.toDF
val invDF = inventory.toDF

val cDF = c2017DF.union(c2016DF).union(c2015DF)

ilsDF.createOrReplaceTempView("ilsDF")
invDF.createOrReplaceTempView("invDF")
cDF.createOrReplaceTempView("cDF")

In [2]:
/*join the inventory data (eg: title, author, etc.) to the checkout table */
val cInvDF = spark.sql("SELECT a.*, b.Title, b.Author, b.PublicationYear, b.Subjects FROM cDF a join invDF b on a.BibNumber = b.BibNum")

In [3]:
cInvDF.printSchema


root
 |-- BibNumber: integer (nullable = true)
 |-- ItemBarcode: long (nullable = true)
 |-- ItemType: string (nullable = true)
 |-- Collection: string (nullable = true)
 |-- CallNumber: string (nullable = true)
 |-- CheckoutDateTime: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- PublicationYear: string (nullable = true)
 |-- Subjects: string (nullable = true)



In [3]:
cInvDF.createOrReplaceTempView("cInvDF")

In [4]:
/* join format data onto main table */
val allDF = spark.sql("SELECT a.*, b.* FROM cInvDF a left join ilsDF b ON a.ItemType = b.Code")

In [5]:
/* filter out duplicates because the joins created a lot of them for some reason */
val allDFDistinct = allDF.distinct
allDFDistinct.count

17647914

In [7]:
allDFDistinct.printSchema

root
 |-- BibNumber: integer (nullable = true)
 |-- ItemBarcode: long (nullable = true)
 |-- ItemType: string (nullable = true)
 |-- Collection: string (nullable = true)
 |-- CallNumber: string (nullable = true)
 |-- CheckoutDateTime: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- PublicationYear: string (nullable = true)
 |-- Subjects: string (nullable = true)
 |-- Code: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Code Type: string (nullable = true)
 |-- Format Group: string (nullable = true)
 |-- Format Subgroup: string (nullable = true)
 |-- Category Group: string (nullable = true)
 |-- Category Subgroup: string (nullable = true)



In [6]:
allDFDistinct.createOrReplaceTempView("allDFDistinct")

In [10]:
/* checkout the format groups */
spark.sql("SELECT `Format Subgroup`, count(*) from allDFDistinct GROUP BY `Format Subgroup`").show()

+---------------+--------+
|Format Subgroup|count(1)|
+---------------+--------+
|     Periodical|      16|
|            Kit|    1948|
|            Art|       2|
|           null|   49469|
|           Book|10565380|
|      Data Disc|      19|
|     Audio Tape|      18|
| Audiobook Disc|      19|
|     Video Disc| 5136268|
|     Audio Disc| 1873157|
|    Music Score|   21618|
+---------------+--------+



In [7]:
/* create year, month, and day columns from the checkoutDateTime variable */
val master = spark.sql("""SELECT *, 
                        SUBSTRING(CheckoutDateTime,3,2) as checkout_day,
                        SUBSTRING(CheckoutDateTime,0,2) as checkout_month,
                        SUBSTRING(CheckoutDateTime,7,4) as checkout_year
                        from allDFDistinct""")

In [8]:
master.createOrReplaceTempView("master")
master.printSchema

root
 |-- BibNumber: integer (nullable = true)
 |-- ItemBarcode: long (nullable = true)
 |-- ItemType: string (nullable = true)
 |-- Collection: string (nullable = true)
 |-- CallNumber: string (nullable = true)
 |-- CheckoutDateTime: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- PublicationYear: string (nullable = true)
 |-- Subjects: string (nullable = true)
 |-- Code: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Code Type: string (nullable = true)
 |-- Format Group: string (nullable = true)
 |-- Format Subgroup: string (nullable = true)
 |-- Category Group: string (nullable = true)
 |-- Category Subgroup: string (nullable = true)
 |-- checkout_day: string (nullable = true)
 |-- checkout_month: string (nullable = true)
 |-- checkout_year: string (nullable = true)



In [13]:
/* check that the years and months were created correctly */
spark.sql("select distinct checkout_year, checkout_month from master").show(50)

+-------------+--------------+
|checkout_year|checkout_month|
+-------------+--------------+
|         2015|            09|
|         2016|            05|
|         2017|            08|
|         2015|            11|
|         2017|            01|
|         2016|            04|
|         2015|            07|
|         2015|            05|
|         2017|            07|
|         2016|            07|
|         2017|            05|
|         2016|            06|
|         2017|            09|
|         2015|            08|
|         2015|            02|
|         2016|            02|
|         2016|            03|
|         2015|            06|
|         2017|            04|
|         2015|            10|
|         2015|            03|
|         2016|            12|
|         2015|            04|
|         2017|            02|
|         2017|            03|
|         2016|            08|
|         2016|            10|
|         2016|            11|
|         2015|            12|
|       

In [14]:
/* I thought it might be interesting to see the breakdown of checkouts by ItemType but it isn't the type of info I thought
it would be. It turns out Category Groups and Subgroups are closer to what I was looking for */
spark.sql("SELECT DISTINCT itemtype FROM master").show(50)

+--------+
|itemtype|
+--------+
|   jcdvd|
|   acart|
|    bccd|
|   arkit|
|  ucunkn|
|   ackit|
|  areqnh|
|    bcbk|
|  aceqnh|
|    accd|
|    ahbk|
| ucunknj|
|   accas|
|   acpam|
|   acmap|
|   bcdvd|
|   acdvd|
| alaptop|
|    acbk|
|   acper|
|    jrbk|
|   ardvd|
|   jccas|
|   acmus|
|  pkbknh|
|  ucfold|
| accdrom|
|    arbk|
| atablet|
|    areq|
|    aceq|
|   arper|
|   jcmus|
|    jcbk|
|   jckit|
|    arcd|
|    jccd|
|  dcillb|
+--------+



In [15]:
/*I thought about looking at popularity of new vs old books and unsurprisingly newer books are much more popular but 
I couldn't figure out how to strip the non-numeric characters out of the column to get a clean read
This will be something to explore later*/
spark.sql("SELECT PublicationYear, count(*) as count from master GROUP BY PublicationYear ORDER BY count DESC").show(50)

+---------------+-------+
|PublicationYear|  count|
+---------------+-------+
|         [2015]|1677762|
|         [2014]|1326865|
|         [2016]|1176306|
|          2015.| 832729|
|          2014.| 714396|
|          2013.| 637012|
|          2016.| 592245|
|          2012.| 560918|
|         [2013]| 482614|
|         c2012.| 476802|
|          2011.| 475915|
|         c2013.| 397488|
|         [2017]| 383794|
|          2010.| 376678|
|         c2011.| 325003|
|         c2010.| 286208|
|          2009.| 266003|
|         c2009.| 248512|
|          2008.| 230569|
|         c2008.| 188489|
|          2017.| 184481|
|          2007.| 179156|
|          2006.| 153095|
|         c2007.| 143783|
|         c2014.| 143347|
|         p2013.| 134306|
|          2005.| 127170|
|         c2015.| 122302|
|         c2006.| 117126|
|         [2012]| 115549|
|         p2012.| 110835|
|         p2011.|  97719|
|         c2005.|  97561|
|         c2004.|  96773|
|          2004.|  94378|
|         c2

In [16]:
/* something I would have liked to do given more time is split this title string into title, studio, producer,
etc. all in their own column instead of a clumsy string of info crammed under the "title" column. This is probably
a result of having a bunch of media types in the same table. By breaking this information out we would either need
to tolerate a lot of missing values (eg: no books would have studios or producers) or break the info into a bunch of
separate tables*/

/* find top 10 movies in each year */
spark.sql("SELECT title, count(*) as count from master WHERE `Format Subgroup`='Video Disc' and checkout_year='2015' GROUP BY title ORDER BY count DESC ").show(10, 100)
spark.sql("SELECT title, count(*) as count from master WHERE `Format Subgroup`='Video Disc' and checkout_year='2016' GROUP BY title ORDER BY count DESC ").show(10, 100)
spark.sql("SELECT title, count(*) as count from master WHERE `Format Subgroup`='Video Disc' and checkout_year='2017' GROUP BY title ORDER BY count DESC ").show(10, 100)

+----------------------------------------------------------------------------------------------------+-----+
|                                                                                               title|count|
+----------------------------------------------------------------------------------------------------+-----+
|Gone girl [videorecording] / Twentieth Century Fox ; Regency ; produced by Arnon Milchan, Reese W...| 4375|
|The imitation game [videorecording] / The Weinstein Company ; Black Bear Pictures ; a Bristol Aut...| 4235|
|Boyhood [videorecording] / IFC Films ; IFC Productions ; a Detour FilmProduction ; producers, Jon...| 4052|
|Birdman, or (the unexpected virtue of ignorance) [videorecording] / Fox Searchlight Pictures ; Re...| 4029|
|The theory of everything [videorecording] / Focus Features ; Working Title ; directed by James Ma...| 3716|
|Interstellar [videorecording] / Paramount Pictures ; Warner Bros. Pictures ; Legendary Pictures ;...| 3489|
|The Grand Budapest

In [17]:
/*
it looks like the top 10 movies from 2015 accounted for more checkouts than the top 10 movies in 2016 or 2017.
is this because people checked out fewer movies overall or that the distribution of movie checkouts was different.
Count the total number of checkouts by year where format subgroup is video disc
*/

Name: Syntax Error.
Message: 
StackTrace: 

In [18]:
/* find total movie checkouts by year to see what proportion is accounted for by the top 10 */

spark.sql("""SELECT checkout_year, count(*) as count FROM master
           WHERE `Format Subgroup`='Video Disc' 
           GROUP BY checkout_year
           ORDER BY checkout_year""").show()

+-------------+-------+
|checkout_year|  count|
+-------------+-------+
|         2015|1928984|
|         2016|1837241|
|         2017|1370043|
+-------------+-------+



In [19]:
/* video discs are becoming less popular over the period and the proportion of checkouts covered by the top 10 movies is
staying relatively stable with the top 10 accounting for 19% in 2015, 18% in 2016, and 17% in 2017
*/

Name: Syntax Error.
Message: 
StackTrace: 

In [20]:
/* find the top 10 most popular books for each year*/
spark.sql("""SELECT title, count(*) as count from master 
           WHERE `Format Subgroup`='Book' and checkout_year='2015' 
           GROUP BY title 
           ORDER BY count DESC""").show(10, 100)

spark.sql("""SELECT title, count(*) as count from master 
             WHERE `Format Subgroup`='Book' and checkout_year='2016' 
             GROUP BY title 
             ORDER BY count DESC """).show(10, 100)

spark.sql("""SELECT title, count(*) as count from master 
             WHERE `Format Subgroup`='Book' and checkout_year='2017' 
             GROUP BY title 
             ORDER BY count DESC """).show(10, 100)
/* similar to the movies, I would like to split the title column up into title, author, etc. to make it easier to 
do more fun and interesting data pestering. Tell me your secrets library!!!!!*/

+----------------------------------------------------------------------------------------------------+-----+
|                                                                                               title|count|
+----------------------------------------------------------------------------------------------------+-----+
|                                                              The girl on the train / Paula Hawkins.| 3333|
|                                              All the light we cannot see : a novel / Anthony Doerr.| 2670|
|The life-changing magic of tidying up : the Japanese art of decluttering and organizing / Marie K...| 2424|
|                                 Being mortal : medicine and what matters in the end / Atul Gawande.| 2025|
|                                                                           Yes please / Amy Poehler.| 1639|
|The boys in the boat : nine Americans and their epic quest for gold at the 1936 Berlin Olympics /...| 1560|
|                  

Name: Syntax Error.
Message: 
StackTrace: 

In [21]:
/* books seem to have more staying power than movies. 4 of the top 10 in 2015 were also in the top 10 in 2016 and 1 from 2016
is in the top 10 in 2017. None of the top 10 movies overlapped*/
master.printSchema

root
 |-- BibNumber: integer (nullable = true)
 |-- ItemBarcode: long (nullable = true)
 |-- ItemType: string (nullable = true)
 |-- Collection: string (nullable = true)
 |-- CallNumber: string (nullable = true)
 |-- CheckoutDateTime: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- PublicationYear: string (nullable = true)
 |-- Subjects: string (nullable = true)
 |-- Code: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Code Type: string (nullable = true)
 |-- Format Group: string (nullable = true)
 |-- Format Subgroup: string (nullable = true)
 |-- Category Group: string (nullable = true)
 |-- Category Subgroup: string (nullable = true)
 |-- checkout_day: string (nullable = true)
 |-- checkout_month: string (nullable = true)
 |-- checkout_year: string (nullable = true)



In [22]:
/* find the number of checkouts by month of books with "clinton" in the title */
spark.sql("""SELECT checkout_year, checkout_month, count(*) as count from master 
           WHERE `Format Subgroup`='Book' and LOWER(title) LIKE '%clinton%' 
           GROUP BY checkout_year, checkout_month
           ORDER BY checkout_year, checkout_month """).show(50, 100)
/* we see a large jump in the popularity of books about the Clinton's in 2016 which is probably mainly due to 
2 factors that feed each other: the 2016 election led to a boost in interest and the boost in interest led to 
a boost in publications which led to more interest...
with more time I would look into the publication years, months of each of the books containing "clinton" and get a 
more granular view of the checkout patterns for each book and how it maps to real world events

I think this project has turned into a scattered data exploration where more questions are created than answered :) */

+-------------+--------------+-----+
|checkout_year|checkout_month|count|
+-------------+--------------+-----+
|         2015|            01|  109|
|         2015|            02|   77|
|         2015|            03|  106|
|         2015|            04|  100|
|         2015|            05|   85|
|         2015|            06|   88|
|         2015|            07|  119|
|         2015|            08|   88|
|         2015|            09|   91|
|         2015|            10|  137|
|         2015|            11|  100|
|         2015|            12|   82|
|         2016|            01|  115|
|         2016|            02|  116|
|         2016|            03|  132|
|         2016|            04|  134|
|         2016|            05|  136|
|         2016|            06|  179|
|         2016|            07|  185|
|         2016|            08|  190|
|         2016|            09|  196|
|         2016|            10|  201|
|         2016|            11|  212|
|         2016|            12|  108|
|

Name: Syntax Error.
Message: 
StackTrace: 

In [23]:
/* find the number of checkouts by month of books with "trump" in the title */
spark.sql("""SELECT checkout_year, checkout_month, count(*) as count from master 
           WHERE `Format Subgroup`='Book' and LOWER(title) LIKE '%trump%' 
           GROUP BY checkout_year, checkout_month
           ORDER BY checkout_year, checkout_month """).show(50, 100)

/* again we see a growth in popularity in trump books as the 2016 election approaches and continued popularity afterwards.
Similarly to Clinton this is probably due to more interest leading to more publishing leading to more interest*/

+-------------+--------------+-----+
|checkout_year|checkout_month|count|
+-------------+--------------+-----+
|         2015|            01|  134|
|         2015|            02|  111|
|         2015|            03|  131|
|         2015|            04|  115|
|         2015|            05|  118|
|         2015|            06|   98|
|         2015|            07|  131|
|         2015|            08|  146|
|         2015|            09|  130|
|         2015|            10|  131|
|         2015|            11|  178|
|         2015|            12|  169|
|         2016|            01|  161|
|         2016|            02|  160|
|         2016|            03|  166|
|         2016|            04|  159|
|         2016|            05|  168|
|         2016|            06|  165|
|         2016|            07|  222|
|         2016|            08|  235|
|         2016|            09|  207|
|         2016|            10|  227|
|         2016|            11|  255|
|         2016|            12|  217|
|

Name: Syntax Error.
Message: 
StackTrace: 

In [25]:
/* find out what is going on with the audio tape checkouts. Do people have cassette players anymore? */
spark.sql("SELECT title FROM master WHERE `Format Subgroup` = 'Audio Tape'").show(20,100)


/* see if the popularity of the library is changing over time */
spark.sql("""SELECT checkout_year, checkout_month, count(*) 
             FROM master 
             GROUP BY checkout_year, checkout_month
             ORDER BY checkout_year, checkout_month DESC""").show(50,100)

+-------------------------------------------------------------------------------------------+
|                                                                                      title|
+-------------------------------------------------------------------------------------------+
|                                The Ditson school and community band series; leader's book.|
|                           Schachnovelle [sound recording] ; Der Amokläufer / Stefan Zweig.|
|Twinkle, twinkle, little star [sound recording] / Read and sung by Iza Trapani and friends.|
|Twinkle, twinkle, little star [sound recording] / Read and sung by Iza Trapani and friends.|
|Twinkle, twinkle, little star [sound recording] / Read and sung by Iza Trapani and friends.|
|Twinkle, twinkle, little star [sound recording] / Read and sung by Iza Trapani and friends.|
|Twinkle, twinkle, little star [sound recording] / Read and sung by Iza Trapani and friends.|
|Twinkle, twinkle, little star [sound recording] / Read and 

In [10]:
/* save master dataframe as csv */
master.write.format("csv").save("file:///data/SeattleLibData")